In [1]:
## All to folder
## generate prompt
# !ls data
# import time
# time.sleep(60*30)

# Try to do:
# torch.cuda.empty_cache()

In [2]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig

def init_lora_model_and_tokenizer(default_model,
                             LORA_R,
                             LORA_ALPHA,
                             LORA_DROPOUT
                            ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer



class MyCustomCallback(TensorBoardCallback):
    #log_bleu_steps_factor = 5
    bleu_generation_max_new_tokens = 30
    bleu_fn_test_data = "temp/t2c_answers.json"
    bleu_fn_etalon = "temp/answers.json"
    log_step = 0
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        super().on_log(args, state, control, logs=logs, **kwargs)
        #print("kwargs", len(kwargs), kwargs.keys())
        if self.tb_writer is not None:
            #print(state)
            #print(state.global_step)
            #print(self.log_step)
            if (self.log_step % self.log_bleu_steps_factor ==0):
                model = kwargs['model']
                tokenizer = kwargs['tokenizer']
                
                model.eval()
                assert not model.training
                generation_config = GenerationConfig(max_new_tokens = self.bleu_generation_max_new_tokens,
                                                     # min_new_tokens = 5,
                                                     temperature = 1.0
                                                    )
                print("generation_config:", generation_config)
                evaluator = EvaluateTestSet(generation_config = generation_config,
                                            fn_test_data = self.bleu_fn_test_data,
                                            fn_etalon = self.bleu_fn_etalon,
                                            batch_size = 1
                                       )

                metric_res = evaluator.evaluate(model=model, 
                                                tokenizer=tokenizer,
                                               )
                model.train()
                assert model.training
                print(metric_res)
                for key, val in metric_res.items():
                    #add "custom/something"
                    self.tb_writer.add_scalar(key, val, state.global_step)
                self.tb_writer.flush()
            self.log_step += 1


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [3]:
CONFIG_PATH = "/root/experiments_config/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v20"
# t2c_concode_220428_v18.json

In [4]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [5]:
experiment_config['resume_from_checkpoint'] = True

In [6]:
assert experiment_config['resume_from_checkpoint'] == True

In [7]:
assert experiment_config['experiment_name'] == experiment_name

In [8]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [9]:
!mkdir {current_experiment_path}

mkdir: cannot create directory ‘/root/experiments/t2c_concode_220428_v20’: File exists


In [10]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [11]:
setattr(MyCustomCallback, "log_bleu_steps_factor", experiment_config['log_bleu_steps_factor'])

In [12]:
MyCustomCallback.log_bleu_steps_factor

50

In [13]:
model, tokenizer = init_lora_model_and_tokenizer(default_model = experiment_config["default_model"],
                                                 LORA_R = experiment_config["LORA_R"],
                                                 LORA_ALPHA = experiment_config["LORA_ALPHA"],
                                                 LORA_DROPOUT = experiment_config["LORA_DROPOUT"]
                                                )


data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-3ac2744fedc77f2f/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
tokenizer_val = LLaMATokenizer.from_pretrained(
    experiment_config['default_model'], add_eos_token=True
)
tokenizer_val.pad_token_id = 0  # unk. we want this to be different from the eos token


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [15]:
# experiment_config["logging_steps"] = 1
# experiment_config["eval_steps"] = 1

In [16]:
experiment_config
# {'experiment_name': 't2c_concode_220428_v19',
#  'fn_train_dataset': '/root/data/t2c_train.json',
#  'fn_eval_dataset': '/root/data/t2c_answers.json',
#  'default_model': 'decapoda-research/llama-7b-hf',
#  'MICRO_BATCH_SIZE': 2,
#  'BATCH_SIZE': 10,
#  'EPOCHS': 2,
#  'LEARNING_RATE': 0.0002,
#  'CUTOFF_LEN': 256,
#  'LORA_R': 16,
#  'LORA_ALPHA': 16,
#  'LORA_DROPOUT': 0.05,
#  'warmup_steps': 200,
#  'fp16': True,
#  'logging_steps': 10,
#  'eval_steps': 100,
#  'evaluation_strategy': 'steps',
#  'save_total_limit': 1,
#  'save_strategy': 'steps',
#  'save_steps': 500,
#  'seed': 42,
#  'logging_strategy': 'steps',
#  'report_to': 'tensorboard',
#  'mlm': False,
#  'truncation': True,
#  'padding': 'max_length',
#  'config_use_cache': False,
#  'resume_from_checkpoint': False,
#  'bleu_batch_size': 5,
#  'GRADIENT_ACCUMULATION_STEPS': 5,
#  'log_bleu_steps_factor': 50}

{'experiment_name': 't2c_concode_220428_v20',
 'fn_train_dataset': '/root/data/t2c_train.json',
 'fn_eval_dataset': '/root/data/t2c_answers.json',
 'default_model': 'decapoda-research/llama-7b-hf',
 'MICRO_BATCH_SIZE': 2,
 'BATCH_SIZE': 10,
 'EPOCHS': 2,
 'LEARNING_RATE': 0.0002,
 'CUTOFF_LEN': 256,
 'LORA_R': 64,
 'LORA_ALPHA': 16,
 'LORA_DROPOUT': 0.05,
 'warmup_steps': 200,
 'fp16': True,
 'logging_steps': 10,
 'eval_steps': 100,
 'evaluation_strategy': 'steps',
 'save_total_limit': 1,
 'save_strategy': 'steps',
 'save_steps': 500,
 'seed': 42,
 'logging_strategy': 'steps',
 'report_to': 'tensorboard',
 'mlm': False,
 'truncation': True,
 'padding': 'max_length',
 'config_use_cache': False,
 'resume_from_checkpoint': True,
 'bleu_batch_size': 5,
 'GRADIENT_ACCUMULATION_STEPS': 5,
 'log_bleu_steps_factor': 50}

In [17]:
# experiment_config["resume_from_checkpoint"]

In [18]:
from prompter import Prompter
prompter = Prompter()

def generate_prompt(data_point):
    if "input" in data_point and data_point["input"]:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        input = data_point["input"],
                                        label = data_point["output"]
                                       )
    else:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        #input = None,
                                        label = data_point["output"]
                                       )

/root/ipynb/prompter/templates/


In [ ]:
# def generate_prompt(data_point):
#     # sorry about the formatting disaster gotta move fast
#     if data_point["input"]:
#         return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Input:
# {data_point["input"]}
# ### Response:
# {data_point["output"]}"""
#     else:
#         return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction:
# {data_point["instruction"]}
# ### Response:
# {data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=experiment_config["truncation"],
        max_length=experiment_config["CUTOFF_LEN"],
        padding=experiment_config["padding"]
    )
)

trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer_val,
    train_dataset=data["train"],
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"],
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"]
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              ),
    callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
600,0.975300,1.120254
700,1.003700,1.114299
800,0.983400,1.112950
900,1.007200,1.111197
1000,0.942500,1.110582
1100,0.982100,1.106141
1200,0.896800,1.107864
1300,0.918900,1.103898
1400,0.900700,1.104232
1500,0.921500,1.102302


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.28.1"
}



...

{'EM': 0.0, 'BLEU': 0.21851380937922585, 'brevity_penalty': 0.6758238404571268, 'ratio': 0.7184822521419829, 'translation_length': 587, 'reference_length': 817, 'precisions_0': 0.5884353741496599, 'precisions_1': 0.3888888888888889, 'precisions_2': 0.26136363636363635, 'precisions_3': 0.1827309236947791}
generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.28.1"
}



100%|████████████████████████████████████████| 30/30 [00:00<00:00, 54708.31it/s]


{'EM': 0.0, 'BLEU': 0.2209089188776829, 'brevity_penalty': 0.6901782584838726, 'ratio': 0.7294981640146879, 'translation_length': 596, 'reference_length': 817, 'precisions_0': 0.592964824120603, 'precisions_1': 0.3880070546737213, 'precisions_2': 0.2569832402234637, 'precisions_3': 0.17751479289940827}
generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.28.1"
}



100%|███████████████████████████████████████| 30/30 [00:00<00:00, 163626.94it/s]


{'EM': 0.0, 'BLEU': 0.22375966226223992, 'brevity_penalty': 0.6629514358413311, 'ratio': 0.7086903304773562, 'translation_length': 579, 'reference_length': 817, 'precisions_0': 0.6189655172413793, 'precisions_1': 0.4163636363636364, 'precisions_2': 0.27115384615384613, 'precisions_3': 0.18571428571428572}


In [ ]:
model.save_pretrained(current_experiment_path)

In [ ]:
# !rm -rf {current_experiment_path}

In [21]:
!df -h .

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   45G   84K 100% /


In [22]:
!ls -lah {current_experiment_path}

total 56K
drwxr-xr-x  6 root root  250 May 10 23:47 .
drwxr-xr-x 13 root root 4.0K May 10 22:36 ..
drwxr-xr-x  2 root root   65 May 10 22:38 1683758323.605443
drwxr-xr-x  2 root root   65 May 10 22:38 1683758323.6125872
drwxr-xr-x  2 root root   31 May 10 23:47 checkpoint-1000
drwxr-xr-x  2 root root  243 May 10 23:14 checkpoint-500
-rw-r--r--  1 root root  22K May 10 23:47 events.out.tfevents.1683758323.8d048d63ed1a.14655.0
-rw-r--r--  1 root root  24K May 10 23:47 events.out.tfevents.1683758323.8d048d63ed1a.14655.2
-rw-r--r--  1 root root  794 May 10 22:36 experiment_config.json


In [23]:
current_experiment_path

'/root/experiments/t2c_concode_220428_v20'

In [28]:
!ls -lah /root/experiments/t2c_concode_220428_v20/checkpoint-500

total 7.0G
drwxr-xr-x 2 root root  243 May 10 23:14 .
drwxr-xr-x 6 root root  250 May 10 23:47 ..
-rw-r--r-- 1 root root 257M May 10 23:14 optimizer.pt
-rw-r--r-- 1 root root 6.7G May 10 23:14 pytorch_model.bin
-rw-r--r-- 1 root root  15K May 10 23:14 rng_state.pth
-rw-r--r-- 1 root root  557 May 10 23:14 scaler.pt
-rw-r--r-- 1 root root  627 May 10 23:14 scheduler.pt
-rw-r--r-- 1 root root  423 May 10 23:14 special_tokens_map.json
-rw-r--r-- 1 root root 489K May 10 23:14 tokenizer.model
-rw-r--r-- 1 root root  714 May 10 23:14 tokenizer_config.json
-rw-r--r-- 1 root root 7.1K May 10 23:14 trainer_state.json
-rw-r--r-- 1 root root 3.5K May 10 23:14 training_args.bin


In [44]:
!ls -lah /root/experiments/t2c_concode_220428_v20/checkpoint-500

total 7.0G
drwxr-xr-x 2 root root  243 May 10 23:14 .
drwxr-xr-x 5 root root  227 May 11 07:54 ..
-rw-r--r-- 1 root root 257M May 10 23:14 optimizer.pt
-rw-r--r-- 1 root root 6.7G May 10 23:14 pytorch_model.bin
-rw-r--r-- 1 root root  15K May 10 23:14 rng_state.pth
-rw-r--r-- 1 root root  557 May 10 23:14 scaler.pt
-rw-r--r-- 1 root root  627 May 10 23:14 scheduler.pt
-rw-r--r-- 1 root root  423 May 10 23:14 special_tokens_map.json
-rw-r--r-- 1 root root 489K May 10 23:14 tokenizer.model
-rw-r--r-- 1 root root  714 May 10 23:14 tokenizer_config.json
-rw-r--r-- 1 root root 7.1K May 10 23:14 trainer_state.json
-rw-r--r-- 1 root root 3.5K May 10 23:14 training_args.bin


In [58]:
# !rm -rf /root/experiments/t2c_concode_220428_v19/checkpoint-20000/
# !df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   32G   14G  71% /


In [ ]:
# !rm -rf {current_experiment_path}/checkpoint-1000

In [ ]:
print(123)

In [ ]:
# import sys
# sys.exit()

In [ ]:
0